In [74]:
def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
    image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training'
    
    TOO_SMALL = 200000
    SMALL = 500000
    MEDIUM = 1200000
    LARGE = 2000000
    
    X_train = []
    Y_train = []
    counter = 1
    
    img_height = len(X) #Usually 128. this might change depending on Annaya & Danielle's input 
    img_width = len(X[0]) #Usually 128. this might change depending on Annaya & Danielle's input 
    epochs = 1 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
    #total_training = total number of training data len()
    batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
    total_images = 1

In [75]:
#pip install opencv-python

In [76]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd


from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt

In [77]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= 200000: #<= TOO_SMALL
        size = 0
    elif resolution <= 500000: #SMALL
        size = 1
    elif resolution <= 1200000: #MEDIUM
        size = 2
    elif resolution <= 2000000:#LARGE
        size = 3
    else:
        size = 6
    return size

In [78]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < 2000000] # < LARGE
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > 200000] #> TOO_SMALL
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    
    return df

In [79]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgRGB, size): #imgLAB imgRGB is fresh images from col
    b,g,r = cv2.split(imgRGB)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        rgb_compressed = cv2.merge(inverted_img)

    return rgb_compressed

In [80]:
# image padding

def pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255]
    MAX = 1888
    
    if height % 2 == 0:
        top = int((MAX - height)/ 2)
        bottom = int((MAX - height)/ 2)
    else: 
        top = int((MAX - (height-1))/ 2)
        bottom = int(((MAX - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX - width)/ 2)
        right = int((MAX - width)/ 2)
    else: 
        left = int((MAX - (width-1))/ 2)
        right = int(((MAX - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [81]:
# The function to preprocess image: by check image category, compress, separate color channels, and pad
# gives out padded images in 2 groups,which are padded Lambda channel (Black&white) and AB channel(colors)

#We can decide to take file path or im collection  or list
#note: we are most familiar with controlling list
n = 1

def prep_img(img_path):
    imgRGB = cv2.imread(img_path)

    #check the image category by 'shape' function
    image_size_category = shape(imgRGB)
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    image_pca = pca_rgb(imgRGB, image_size_category) 
 
    #image_pca.astype("float32")/ 255 
    image_pca = np.float32(1.0/255*image_pca)
    imgLAB = cv2.cvtColor(image_pca, cv2.COLOR_RGB2Lab)
    
    #print("image converted to LAB")
    padded_image = pad(imgLAB)
    
    #separate L and AB color channels
    Y = padded_image[:, :,1:]
    X = padded_image[:, :, 0]
    
    Y = Y / 128
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1)
    Y = Y.reshape(1, 1888, 1888, 2)
    
    #for index, item in enumerate(iterable):
    #print(index, item)
    
    print("image processed")

    return Y, X 

In [82]:
#def prep_dataframe(df):
    #for count, index, row in enumerate(df.iterrows()): 
        #Y, X = prep_img(row["image_path"])
        #Y_train.append(Y)
        #X_train.append(X)
        #print("Processed", count, "images for upload")
#enumerate function is in progress

In [83]:
 image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training'

In [84]:
def make_data():
    #TODO make it work with init() 
    df = create_dataframe(image_folder)
    #call prep_img() which return Y,X
    X_list = []
    Y_list = []
    for i in df['image_path']:
        Y_prep, X_prep = prep_img(i) # another choice is to name a vari
    #append.X (ref with path in the df) to a list of  X images array
    #append.Y to another list of Y images array          
        X_list.append(X_prep)
        Y_list.append(Y_prep)
    #train test val split
    X_train, X_test, Y_train, Y_test = train_test_split(X_list, Y_list, test_size=0.2, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
    print(len(X_train), len(Y_train))
    print(len(X_val), len(Y_val))
    print(len(X_test), len(Y_test))
  
    print("Processed", "images for upload")
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

# Use the code below to process ALL photos and feed them to the model!!!

In [86]:
#def preprocessing ():
    #init
    #df = create_dataframe(image_folder)
    #prep_dataframe(df)
    #return df

In [87]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = make_data()

processing 151 photos for training
dropped  10 photos because they were too large to process
dropped  5 photos because they were too small to process
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
image processed
im

In [88]:
def mode(): 
    model1 = Sequential([
        #encoder
        Conv2D(128, 3, strides = 2, padding = "same", activation = "relu", input_shape= (1888, 1888, 1)), #plug in the shape value #(img_height, img_width, 1)  
        Conv2D(256, 3, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(256, 3, strides = 2, padding = "same", activation = "relu"), 

        #decoder
        Conv2D(256, 3, padding = "same", activation = "relu"), 
        MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
        UpSampling2D(2),
        Conv2D(128, 3, padding = "same", activation = "relu"), 
        UpSampling2D(2), 
        Conv2D(2, 3, padding = "same", activation = "tanh"),
        UpSampling2D(2),
        #this is just trying to add something, see if the error changes
        Dense(32, activation='softmax'),
        Dense(4, activation='softmax'),
        Dense(2, activation='softmax')
    ])
    model1.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model1

In [89]:
model1 = mode()
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 944, 944, 128)     1280      
                                                                 
 conv2d_19 (Conv2D)          (None, 472, 472, 256)     295168    
                                                                 
 conv2d_20 (Conv2D)          (None, 236, 236, 256)     590080    
                                                                 
 conv2d_21 (Conv2D)          (None, 236, 236, 256)     590080    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 236, 236, 256)    0         
 2D)                                                             
                                                                 
 up_sampling2d_9 (UpSampling  (None, 472, 472, 256)    0         
 2D)                                                  

In [90]:
#the following copy from EmiilWalner https://emilwallner.medium.com/colorize-b-w-photos-with-a-100-line-neural-network-53d9b4449f8d
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

In [91]:
# Generate training data
batch_size = 3
def image_a_b_gen(batch_size):
    for batch in datagen.flow(X_train, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)
#We use the images from our folder, Xtrain, to generate images based on the settings above. 
#Then, we extract the black and white layer for the X_batch and the two colors for the two color layers.

In [92]:
model.fit_generator(image_a_b_gen(batch_size), steps_per_epoch=1, epochs=1000)

NameError: name 'model' is not defined

In [ ]:
epochs = 10

In [94]:
history = model1.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2

Epoch 1/10
28/28 - 271s - loss: 52364.6016 - accuracy: 0.0156 - 271s/epoch - 10s/step
Epoch 2/10
28/28 - 240s - loss: 52364.5039 - accuracy: 0.0156 - 240s/epoch - 9s/step
Epoch 3/10
28/28 - 248s - loss: 52364.5039 - accuracy: 0.0156 - 248s/epoch - 9s/step
Epoch 4/10
28/28 - 254s - loss: 52364.5039 - accuracy: 0.0156 - 254s/epoch - 9s/step
Epoch 5/10
28/28 - 239s - loss: 52364.5039 - accuracy: 0.0156 - 239s/epoch - 9s/step
Epoch 6/10
28/28 - 253s - loss: 52364.4883 - accuracy: 0.0156 - 253s/epoch - 9s/step
Epoch 7/10
28/28 - 239s - loss: 52364.4688 - accuracy: 0.0156 - 239s/epoch - 9s/step
Epoch 8/10
28/28 - 220s - loss: 52364.4805 - accuracy: 0.0156 - 220s/epoch - 8s/step
Epoch 9/10
28/28 - 210s - loss: 52364.4883 - accuracy: 0.0156 - 210s/epoch - 7s/step
Epoch 10/10
28/28 - 209s - loss: 52364.4805 - accuracy: 0.0156 - 209s/epoch - 7s/step


In [93]:
train_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
import joblib

filename = 'finalized_model.sav'
joblib.dump(history, filename)
 

#loaded_model = joblib.load(filename)
#result = loaded_model.score(X_test, Y_test)
#print(result)
